In [4]:
import requests as reqs
import json
import pandas as pd

In [5]:
def dataIntialization():
    #On 4/1/2021 was taking a while to complete the response...
    #payment plans request would sometimes time out!
    try:
        debtRequest = reqs.get("https://my-json-server.typicode.com/druska/trueaccord-mock-payments-api/debts")
        debt_dict = json.loads(debtRequest.text)
        debts_df = pd.DataFrame.from_dict(debt_dict)
        
        print("request 1 completed")

        plansReq = reqs.get("https://my-json-server.typicode.com/druska/trueaccord-mock-payments-api/payment_plans")
        payment_plans = json.loads(plansReq.text)
        plans_df = pd.DataFrame.from_dict(payment_plans)

        print("request 2 completed")

        paymentsReq = reqs.get("https://my-json-server.typicode.com/druska/trueaccord-mock-payments-api/payments")
        payments = json.loads(paymentsReq.text)
        payment_df = pd.DataFrame.from_dict(payments)

        print("request 3 completed")

        return (debts_df, plans_df, payment_df)
 
    except:
        print("error?")

In [11]:
def isEnrolled(debt_df, plan_df):
    isEnrolled = []
    paymentPlan_id = []

    for x in debt_df['id']:
        isEnrolled.append(True) if (x in plan_df['debt_id']) else isEnrolled.append(False)
        paymentPlan_id.append(plan_df['id']);

#     debt_df["plan_id"] = paymentPlan_id
    print(paymentPlan_id)
    debt_df["is_in_payment_plan"] = isEnrolled
    
    return debt_df


In [7]:
#So if they are enrolled in a payment plan, instead of using the payments, just use the payment plan amount?
def remainingAmount(debt_df, plan_df, payment_df):
    plan_dict = {}
    for index, row in plan_df.iterrows():
        if (row['id'] in plan_dict):
            plan_dict[row['id']] += row['amount_to_pay']
        else:
            plan_dict[row['id']] = row['amount_to_pay']

    payments_dict = {}
    for index, row in payment_df.iterrows():
        if (row['id'] in plan_dict):
            plan_dict[row['id']] += row['amount_to_pay']
        else:
            plan_dict[row['id']] = row['amount_to_pay']

    for index, row in debt_df.iterrows():
        if (row['is_in_payment_plan'] == True):
            targetId = row['id']
        else:
            pass

In [10]:
def main():
    try:
        debts, plans, payments = dataIntialization()
        debts = isEnrolled(debts, plans)
        print(debts)
        remainingAmount(debts, plans, payments)
    except:
        print("requests didn't load in")

main()

request 1 completed
request 2 completed
request 3 completed
     amount  id  is_in_payment_plan
0    123.46   0                True
1    100.00   1                True
2   4920.34   2                True
3  12938.00   3                True
4   9238.02   4               False
requests didn't load in
